In [2]:
import os

os.environ['LANGSMITH_TRACING_V2'] = 'true'

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.chat_models import ChatTongyi  # noqa: F401
from langchain_community.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_ollama import OllamaEmbeddings

In [4]:
# 自定义的文本拆分器


text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20, separators=['第'])

loader = TextLoader('data/rag/供电规则.txt', encoding='utf-8')
docs = loader.load_and_split(text_splitter=text_splitter)
# vectorstore = Chroma.from_documents(documents=docs, embedding=OllamaEmbeddings(model='bge-m3:latest'), persist_directory='data/rag/knowledge_base')
vectorstore = Chroma(persist_directory='data/rag/knowledge_base', embedding_function=OllamaEmbeddings(model='bge-m3:latest'))
retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 3})

In [5]:
examples = [
    {
        'question': '用户单相用电设备总容量 12 千瓦以下时，可以采用多少供电？',
        'doc': '《供电营业规则》',
        'articles': '第九条',
        'answer': '当用户单相用电设备总容量为12千瓦以下时，可以采用低压220伏的供电方式',
        'link': '127.0.0.1:8000/knowbase/1.pdf',
    },
    {
        'question': '申请非永久性减容的，时长是多久？',
        'doc': '《供电营业规则》',
        'articles': '第二十五条',
        'answer': '申请非永久性减容的，减容次数不受限制，每次减容时长不得少于十五日，最长不得超过两年',
        'link': '127.0.0.1:8000/knowbase/2.pdf',
    },
    {
        'question': '用户集资建设的供电站，建成运营前，由谁管理？',
        'doc': '《供电营业规则》',
        'articles': '第四十九条',
        'answer': '用户集资建设的供电设施建成后，其运行维护管理按照以下规定确定：（一）属于公用性质或占用公用线路规划走廊的，由供电企业统一管理',
        'link': '127.0.0.1:8000/knowbase/3.pdf',
    },
]
example_prompt = ChatPromptTemplate(
    [
        ('human', '{question}'),
        ('ai', '根据{doc}，{articles}。{answer}。[链接]({link})'),
    ]
)

few_shot_promppt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate(
    [
        ('system', '你是供电局客服，根据知识库，模仿以下对话案例回答用户的问题。'),
        few_shot_promppt,
        ('human', '{question}'),
        ('ai', '这是知识库。{docs}'),
    ]
)


In [ ]:
from pathlib import Path

service_ip = '127.0.0.1:8000/'


def format_docs(docs):
    import json

    doc_res = []
    for doc in docs:
        d = {}
        d['name'] = Path(doc.metadata['source']).stem
        d['link'] = service_ip + str(Path('/knowledge_base', Path(doc.metadata['source']).stem).with_suffix('.pdf').as_posix())
        print(d['link'])
        d['content'] = doc.page_content
        doc_res.append(json.dumps(d, ensure_ascii=False))
    return '\n\n'.join(doc_res)


model = ChatTongyi(model='qwen-max')
# model2 = ChatOllama(model='qwen2.5:3b')
rag_chain = RunnableParallel(question=RunnablePassthrough(), docs=retriever | format_docs) | final_prompt | model | StrOutputParser()

In [ ]:
Path('rag', '21')

WindowsPath('rag/21')

In [8]:
def query_knowledge_base(ip, question):
    global service_ip
    service_ip = ip
    for i in rag_chain.stream(question):
        yield {'type': 'table', 'content': i}

In [9]:
for i in query_knowledge_base('127.0.0.1:8000', '发生什么事故，需要告知供电企业？'):
    print(i)

{'type': 'table', 'content': '根据'}
{'type': 'table', 'content': '《供电规则》'}
{'type': 'table', 'content': '第六十五条，发生'}
{'type': 'table', 'content': '以下事故时需要'}
{'type': 'table', 'content': '告知供电企业：\n\n-'}
{'type': 'table', 'content': ' 人身触电死亡；\n'}
{'type': 'table', 'content': '- 导致电力系统'}
{'type': 'table', 'content': '停电；\n- 专线掉'}
{'type': 'table', 'content': '闸或全厂停电；\n'}
{'type': 'table', 'content': '- 电气火灾；\n-'}
{'type': 'table', 'content': ' 重要或大型电气设备'}
{'type': 'table', 'content': '损坏；\n- 停'}
{'type': 'table', 'content': '电期间向电力系统倒'}
{'type': 'table', 'content': '送电。\n\n如果'}
{'type': 'table', 'content': '遇到以上情况，请'}
{'type': 'table', 'content': '及时与我们联系。['}
{'type': 'table', 'content': '链接](127.'}
{'type': 'table', 'content': '0.0.1:'}
{'type': 'table', 'content': '8000/knowledge'}
{'type': 'table', 'content': '_base/供电规则.pdf)'}
{'type': 'table', 'content': ''}
